In [104]:
%pip install -r requirements.txt

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1558 sha256=5dcc9c69254b2317106af0a883ef6b1bb182269e6eaeb38acbad832675ee60d9
  Stored in directory: c:\users\zalipski\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\6a\d4\2e\22908853a465dbeae5d67583a77bacaa0aba24288e7778f840
Successfully built timedelta
Note: you may need to restart the kernel to use updated packages.


# Imports

In [233]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [234]:
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 1000)

In [235]:
match_h1 = pd.read_csv(r"handball_sample\flensburg_kiel_md5_h1.csv", sep = ";")
match_h2 = pd.read_csv(r"handball_sample\flensburg_kiel_md5_h1.csv", sep = ";")
tags = pd.read_csv(r"handball_sample\tags_flensburg_kiel_md5_s2324.csv", sep = ";")

In [236]:
tags.tail()

,# time (in ms),tag text,player
791,1231733,no_possession,NaN
792,1232248,71_elias_eliefsen_á_skipagotu,NaN
793,1234836,no_possession,NaN
794,1235330,7_magnus_landin_jacobsen,NaN
795,1251879,28_eduardo_gurbindo_martinez,NaN


# Data Understanding

In [237]:
# see where multiple players get possession at the same time
match_h1 = match_h1.loc[:, ~match_h1.columns.isin(["heart rate in bpm", "core temperature in celsius", "player orientation in deg",  "Unnamed: 23"])]
match_h1_ball_poss = match_h1[~np.isnan(match_h1["ball possession (id of possessed ball)"])]
dups = match_h1_ball_poss[match_h1_ball_poss.duplicated(["ts in ms", "ball possession (id of possessed ball)"], keep = False)]
counts = dups.groupby(["ts in ms", "ball possession (id of possessed ball)"]).size()
rows_with_three_or_more = counts[counts >= 3].reset_index()
result = match_h1_ball_poss.merge(rows_with_three_or_more, on=["ts in ms", "ball possession (id of possessed ball)"])
result.head(5)

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load,0
0,1694111499950,07.09.2023 20:31:39.950,50453,1833,2,Simon Pytlick,1560268,1,SG Flensburg-Handewitt,-6.084,-9.314,1.6,0.915,-139.537,0.484,83.772,4.344,Backward,1208.0,2.915,3
1,1694111499950,07.09.2023 20:31:39.950,43609,212,24,Jim Gottfridsson,345559,1,SG Flensburg-Handewitt,-5.777,-0.768,1.6,0.831,-119.434,-0.207,29.013,2.667,Right,1208.0,2.970,3
2,1694111499950,07.09.2023 20:31:39.950,43652,1809,33,Kay Smits,1028401,1,SG Flensburg-Handewitt,-8.213,9.140,1.6,0.783,159.438,-0.992,65.081,1.536,Backward,1208.0,1.707,3
3,1694111500000,07.09.2023 20:31:40.000,50453,1833,2,Simon Pytlick,1560268,1,SG Flensburg-Handewitt,-6.114,-9.332,1.6,0.915,-149.564,0.706,83.772,4.877,Backward,1208.0,2.959,3
4,1694111500000,07.09.2023 20:31:40.000,43609,212,24,Jim Gottfridsson,345559,1,SG Flensburg-Handewitt,-5.780,-0.782,1.6,0.830,-116.490,-0.164,29.013,2.728,Right,1208.0,3.042,3


In [238]:
match_h1[5237:].head(5)

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load
5237,1694111447600,07.09.2023 20:30:47.600,43632,1522,26,Johan Hansen,1340010,1,SG Flensburg-Handewitt,15.210,7.471,1.6,0.228,90.987,0.235,10.338,0.867,Left,1208.0,0.378
5238,1694111447600,07.09.2023 20:30:47.600,43679,1548,21,Eric Johansson,1560210,2,THW Kiel,10.065,8.668,1.6,0.337,NaN,0.292,12.361,1.266,NaN,NaN,0.650
5239,1694111447600,07.09.2023 20:30:47.600,50453,1833,2,Simon Pytlick,1560268,1,SG Flensburg-Handewitt,10.926,-3.472,1.6,0.264,NaN,-0.106,7.121,0.895,NaN,NaN,0.707
5240,1694111447600,07.09.2023 20:30:47.600,50455,1106,31,Emil Jakobsen,1637708,1,SG Flensburg-Handewitt,15.000,-6.303,1.6,0.721,116.117,-0.021,10.598,2.567,Left,NaN,0.720
5241,1694111447600,07.09.2023 20:30:47.600,43670,342,7,Magnus Landin Jacobsen,1387507,2,THW Kiel,13.724,9.609,1.6,0.176,NaN,-0.233,16.310,0.697,NaN,1208.0,0.812


In [239]:
match_h2 = match_h2.loc[:, ~match_h2.columns.isin(["heart rate in bpm", "core temperature in celsius", "player orientation in deg",  "Unnamed: 23"])]
match_h2.head(5)

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load
0,1694111429700,07.09.2023 20:30:29.700,43591,214,4,Johannes Golla,876390,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,1694111429700,07.09.2023 20:30:29.700,43603,1104,20,Kevin Møller,592178,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,1694111429700,07.09.2023 20:30:29.700,43632,1522,26,Johan Hansen,1340010,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,1694111429700,07.09.2023 20:30:29.700,43679,1548,21,Eric Johansson,1560210,2,THW Kiel,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,1694111429700,07.09.2023 20:30:29.700,50453,1833,2,Simon Pytlick,1560268,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [240]:
print(match_h1.info())
print(match_h2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545529 entries, 0 to 545528
Data columns (total 20 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   ts in ms                                                  545529 non-null  int64  
 1   formatted local time                                      545529 non-null  object 
 2   sensor id                                                 545529 non-null  int64  
 3   mapped id                                                 545529 non-null  int64  
 4   number                                                    545529 non-null  int64  
 5   full name                                                 545529 non-null  object 
 6   league id                                                 545529 non-null  object 
 7   group id                                                  545529 non-null  int64  
 8   grou

In [241]:
match_h1.describe()

,ts in ms,sensor id,mapped id,number,group id,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,ball possession (id of possessed ball),acceleration load
count,5.455290e+05,545529.000000,545529.000000,545529.000000,545529.000000,545358.000000,545358.000000,545358.000000,544094.000000,452700.000000,538234.000000,545529.000000,497805.000000,28545.000000,505073.000000
mean,1.694113e+12,64605.790788,976.726306,25.363128,1.610769,0.423984,-0.270817,2.093369,1.514201,-1.202580,-0.010898,822.641982,5.225196,1208.255211,1.577174
std,7.516120e+05,70729.109181,615.945067,24.625601,0.621249,12.497399,5.610965,3.049818,1.957923,106.405073,5.988818,602.549870,7.326730,0.435987,1.313736
min,1.694111e+12,43344.000000,207.000000,1.000000,1.000000,-32.079000,-11.948000,0.000000,0.000000,-179.999000,-165.598000,0.000000,0.000000,1208.000000,0.014000
25%,1.694112e+12,43632.000000,335.000000,7.000000,1.000000,-11.885000,-4.638000,1.600000,0.435000,-87.650000,-0.304000,324.972000,1.432000,1208.000000,0.650000
50%,1.694113e+12,43674.000000,1106.000000,20.000000,2.000000,1.198000,-0.069000,1.600000,0.880000,-0.878000,-0.010000,710.553000,2.825000,1208.000000,0.981000
75%,1.694113e+12,43690.000000,1547.000000,31.000000,2.000000,12.247000,4.071750,1.600000,1.689000,82.930000,0.279000,1227.512000,5.480000,1209.000000,2.297000
max,1.694114e+12,314557.000000,1833.000000,99.000000,3.000000,29.673000,12.271000,20.000000,33.315000,180.000000,195.219000,3433.513000,78.040000,1209.000000,10.951000


In [242]:
match_h2.describe()

,ts in ms,sensor id,mapped id,number,group id,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,ball possession (id of possessed ball),acceleration load
count,5.455290e+05,545529.000000,545529.000000,545529.000000,545529.000000,545358.000000,545358.000000,545358.000000,544094.000000,452700.000000,538234.000000,545529.000000,497805.000000,28545.000000,505073.000000
mean,1.694113e+12,64605.790788,976.726306,25.363128,1.610769,0.423984,-0.270817,2.093369,1.514201,-1.202580,-0.010898,822.641982,5.225196,1208.255211,1.577174
std,7.516120e+05,70729.109181,615.945067,24.625601,0.621249,12.497399,5.610965,3.049818,1.957923,106.405073,5.988818,602.549870,7.326730,0.435987,1.313736
min,1.694111e+12,43344.000000,207.000000,1.000000,1.000000,-32.079000,-11.948000,0.000000,0.000000,-179.999000,-165.598000,0.000000,0.000000,1208.000000,0.014000
25%,1.694112e+12,43632.000000,335.000000,7.000000,1.000000,-11.885000,-4.638000,1.600000,0.435000,-87.650000,-0.304000,324.972000,1.432000,1208.000000,0.650000
50%,1.694113e+12,43674.000000,1106.000000,20.000000,2.000000,1.198000,-0.069000,1.600000,0.880000,-0.878000,-0.010000,710.553000,2.825000,1208.000000,0.981000
75%,1.694113e+12,43690.000000,1547.000000,31.000000,2.000000,12.247000,4.071750,1.600000,1.689000,82.930000,0.279000,1227.512000,5.480000,1209.000000,2.297000
max,1.694114e+12,314557.000000,1833.000000,99.000000,3.000000,29.673000,12.271000,20.000000,33.315000,180.000000,195.219000,3433.513000,78.040000,1209.000000,10.951000


# Checking if timestamps in video and tabular data are comparable

## Find unique players of both teams in the game to create the keys txt file

In [243]:
match_complete = pd.concat([match_h1, match_h2])

In [244]:
keys_dict = {"league_id": [], "team": [], "jersey_number": [], "name": []}
for id in match_complete["league id"].unique():
    if "Ball" not in id:
        row = match_complete[match_complete["league id"] == id].iloc[0]
        keys_dict["league_id"].append(row["league id"])
        keys_dict["team"].append(row["group name"])
        keys_dict["jersey_number"].append(row["number"])
        keys_dict["name"].append(row["full name"])

keys_df = pd.DataFrame(keys_dict)

In [245]:
keys_df.sort_values(["team", "jersey_number"], ascending = [True, True])

,league_id,team,jersey_number,name
20,1028491,SG Flensburg-Handewitt,1,Benjamin Buric
4,1560268,SG Flensburg-Handewitt,2,Simon Pytlick
0,876390,SG Flensburg-Handewitt,4,Johannes Golla
1,592178,SG Flensburg-Handewitt,20,Kevin Møller
17,604490,SG Flensburg-Handewitt,22,Mads Mensah Larsen
15,345559,SG Flensburg-Handewitt,24,Jim Gottfridsson
19,1515763,SG Flensburg-Handewitt,25,Lukas Joergensen
2,1340010,SG Flensburg-Handewitt,26,Johan Hansen
21,1611730,SG Flensburg-Handewitt,27,Aksel Horgen
5,1637708,SG Flensburg-Handewitt,31,Emil Jakobsen


## Check time

In [246]:
match_h1["formatted local time"] = pd.to_datetime(match_h1["formatted local time"])

In [247]:
match_h1_start = match_h1[match_h1[~np.isnan(match_h1["ball possession (id of possessed ball)"])].index[0]:]

In [248]:
match_h1_start_poss = match_h1_start.dropna(subset=['ball possession (id of possessed ball)'])

# Drop consecutive rows where the same player is still in possession
# This is achieved by comparing each row with the previous row and keeping only the rows where the player in possession changes
match_h1_start_poss = match_h1_start_poss[match_h1_start_poss['sensor id'] != match_h1_start_poss['sensor id'].shift(1)]

In [249]:
match_h1_start_poss[:50]

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load
37,1694111429800,2023-07-09 20:30:29.800,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-1.954,0.053,1.6,0.410,NaN,NaN,0.000,NaN,NaN,1208.0,0.644
638,1694111431950,2023-07-09 20:30:31.950,43672,327,6,Harald Reinkind,604492,2,THW Kiel,1.762,-6.694,1.6,1.396,0.078,-0.103,1.837,4.734,Forward,1208.0,0.593
2889,1694111439750,2023-07-09 20:30:39.750,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,8.773,0.857,1.6,0.970,160.611,0.195,12.458,3.843,Backward,1208.0,1.070
3528,1694111441900,2023-07-09 20:30:41.900,43679,1548,21,Eric Johansson,1560210,2,THW Kiel,10.076,8.952,1.6,0.775,NaN,-0.438,12.361,2.014,NaN,1208.0,0.662
4274,1694111444350,2023-07-09 20:30:44.350,43652,1809,33,Kay Smits,1028401,1,SG Flensburg-Handewitt,13.380,5.806,1.6,0.162,NaN,-0.075,10.793,0.559,NaN,1208.0,0.288
4911,1694111446500,2023-07-09 20:30:46.500,43670,342,7,Magnus Landin Jacobsen,1387507,2,THW Kiel,13.663,9.898,1.6,0.119,NaN,0.176,16.310,0.461,NaN,1208.0,0.284
5237,1694111447600,2023-07-09 20:30:47.600,43632,1522,26,Johan Hansen,1340010,1,SG Flensburg-Handewitt,15.210,7.471,1.6,0.228,90.987,0.235,10.338,0.867,Left,1208.0,0.378
5241,1694111447600,2023-07-09 20:30:47.600,43670,342,7,Magnus Landin Jacobsen,1387507,2,THW Kiel,13.724,9.609,1.6,0.176,NaN,-0.233,16.310,0.697,NaN,1208.0,0.812
5252,1694111447650,2023-07-09 20:30:47.650,43632,1522,26,Johan Hansen,1340010,1,SG Flensburg-Handewitt,15.201,7.474,1.6,0.245,89.852,0.250,10.338,0.900,Left,1208.0,0.397
5256,1694111447650,2023-07-09 20:30:47.650,43670,342,7,Magnus Landin Jacobsen,1387507,2,THW Kiel,13.727,9.609,1.6,0.145,NaN,-0.233,16.310,0.600,NaN,1208.0,0.789


In [250]:
match_h1_start["time diff from start"] = match_h1_start["formatted local time"] - match_h1_start["formatted local time"].iloc[0]

C:\Users\Zalipski\AppData\Local\Temp\ipykernel_20356\2731903367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_h1_start["time diff from start"] = match_h1_start["formatted local time"] - match_h1_start["formatted local time"].iloc[0]


In [251]:
match_h1_start[(match_h1_start["time diff from start"].dt.microseconds % 500000 == 0) & (~np.isnan(match_h1_start["ball possession (id of possessed ball)"]))].head(10)

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load,time diff from start
37,1694111429800,2023-07-09 20:30:29.800,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-1.954,0.053,1.6,0.410,NaN,NaN,0.000,NaN,NaN,1208.0,0.644,0 days 00:00:00
177,1694111430300,2023-07-09 20:30:30.300,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-1.754,0.189,1.6,0.643,0.794,NaN,0.000,NaN,Forward,1208.0,0.858,0 days 00:00:00.500000
317,1694111430800,2023-07-09 20:30:30.800,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-1.312,0.056,1.6,1.133,-7.303,0.794,0.624,5.701,Forward,1208.0,1.046,0 days 00:00:01
457,1694111431300,2023-07-09 20:30:31.300,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-0.634,0.033,1.6,1.435,-7.368,0.328,0.624,6.187,Forward,1208.0,1.152,0 days 00:00:01.500000
736,1694111432300,2023-07-09 20:30:32.300,43672,327,6,Harald Reinkind,604492,2,THW Kiel,2.244,-6.619,1.6,1.322,4.795,-0.265,1.837,4.128,Forward,1208.0,0.675,0 days 00:00:02.500000
876,1694111432800,2023-07-09 20:30:32.800,43672,327,6,Harald Reinkind,604492,2,THW Kiel,2.853,-6.584,1.6,1.123,9.220,-0.209,3.174,3.602,Forward,1208.0,0.758,0 days 00:00:03
1016,1694111433300,2023-07-09 20:30:33.300,43672,327,6,Harald Reinkind,604492,2,THW Kiel,3.346,-6.420,1.6,1.114,6.655,-0.061,3.174,3.879,Forward,1208.0,0.780,0 days 00:00:03.500000
1156,1694111433800,2023-07-09 20:30:33.800,43672,327,6,Harald Reinkind,604492,2,THW Kiel,3.965,-6.455,1.6,1.058,-2.102,-0.026,4.301,3.674,Forward,1208.0,0.775,0 days 00:00:04
1296,1694111434300,2023-07-09 20:30:34.300,43672,327,6,Harald Reinkind,604492,2,THW Kiel,4.433,-6.459,1.6,1.086,-2.698,0.253,4.301,4.491,Forward,1208.0,0.683,0 days 00:00:04.500000
1436,1694111434800,2023-07-09 20:30:34.800,43672,327,6,Harald Reinkind,604492,2,THW Kiel,4.990,-6.503,1.6,1.320,-6.787,0.276,4.301,5.570,Forward,1208.0,0.607,0 days 00:00:05


In [252]:
match_h1_start[500:1000]

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load,time diff from start
537,1694111431600,2023-07-09 20:30:31.600,50455,1106,31,Emil Jakobsen,1637708,1,SG Flensburg-Handewitt,14.640,-5.856,1.600,0.972,-151.772,-0.170,0.948,3.204,Backward,NaN,0.889,0 days 00:00:01.800000
538,1694111431600,2023-07-09 20:30:31.600,43670,342,7,Magnus Landin Jacobsen,1387507,2,THW Kiel,1.644,8.669,1.600,1.590,-5.546,0.428,0.594,7.034,Forward,NaN,0.691,0 days 00:00:01.800000
539,1694111431600,2023-07-09 20:30:31.600,314557,1208,99,Ball2 Ball,FlensBall2,3,Ball,-0.022,-0.305,2.267,1.796,NaN,3.416,0.539,NaN,NaN,NaN,NaN,0 days 00:00:01.800000
540,1694111431600,2023-07-09 20:30:31.600,43672,327,6,Harald Reinkind,604492,2,THW Kiel,1.245,-6.694,1.600,1.400,-2.120,0.267,0.609,5.835,Forward,NaN,0.623,0 days 00:00:01.800000
541,1694111431600,2023-07-09 20:30:31.600,46783,338,53,Nikola Bilyk,1029445,2,THW Kiel,-0.158,0.029,1.600,1.441,-9.545,0.096,0.624,5.454,Forward,1208.0,1.148,0 days 00:00:01.800000
542,1694111431600,2023-07-09 20:30:31.600,43659,1811,43,Blaz Blagotinsek,797318,1,SG Flensburg-Handewitt,12.361,-0.783,1.600,0.354,-176.651,0.095,0.000,1.344,Backward,NaN,1.083,0 days 00:00:01.800000
543,1694111431600,2023-07-09 20:30:31.600,43677,333,17,Patrick Wiencek,125232,2,THW Kiel,2.954,-0.095,1.600,2.690,-4.786,0.318,1.283,11.447,Forward,NaN,3.100,0 days 00:00:01.800000
544,1694111431600,2023-07-09 20:30:31.600,43674,1546,16,Tomas Mrkva,124259,2,THW Kiel,-7.239,-0.472,1.600,1.143,26.750,-0.055,1.084,3.999,Forward,NaN,0.588,0 days 00:00:01.800000
545,1694111431600,2023-07-09 20:30:31.600,43652,1809,33,Kay Smits,1028401,1,SG Flensburg-Handewitt,11.835,2.914,1.600,0.412,NaN,0.266,0.000,1.715,NaN,NaN,1.158,0 days 00:00:01.800000
546,1694111431650,2023-07-09 20:30:31.650,43591,214,4,Johannes Golla,876390,1,SG Flensburg-Handewitt,12.372,1.616,1.600,0.352,NaN,0.074,0.000,1.322,NaN,NaN,1.026,0 days 00:00:01.850000


### Convert tags timestamp to actual time

In [253]:
tags["# time (in ms)"] = tags["# time (in ms)"] - tags["# time (in ms)"].iloc[0]

# Get the game's start time from df1
game_start_time = match_h1_start_poss['formatted local time'].iloc[0]

# Convert milliseconds in df2 to the datetime format
tags['adjusted_timestamp'] = tags['# time (in ms)'].apply(lambda x: game_start_time + timedelta(milliseconds=x + 500))

# Round the timestamps to nearest 1/20 second (50 milliseconds)
def round_to_nearest_50ms(dt):
    # Convert microseconds to milliseconds and round to nearest 50 milliseconds
    milliseconds = round(dt.microsecond / 1000.0 / 50.0) * 50
    # Rebuild the datetime with the new milliseconds, adjusting for overflow
    new_second = dt.second
    if milliseconds >= 1000:
        milliseconds -= 1000
        new_second += 1
    # Ensure the seconds don't overflow
    if new_second >= 60:
        new_second -= 60
        dt += timedelta(minutes=1)
    # Return the rounded datetime
    return dt.replace(second=new_second, microsecond=int(milliseconds * 1000))

tags.drop(["player"], axis = 1, inplace = True)

tags = tags[tags["tag text"] != tags["tag text"].iloc[1]]

tags['rounded_timestamp'] = tags['adjusted_timestamp'].apply(round_to_nearest_50ms)

In [254]:
game_start_time

Timestamp('2023-07-09 20:30:29.800000')

In [255]:
# Überprüfung Zeit, dieser Wert stellt Uhrzeit in Video dar, als Ball niedrigsten Punkt erreichte, 2023-07-09 20:31:01.014000
game_start_time + timedelta(milliseconds = 31214)

Timestamp('2023-07-09 20:31:01.014000')

In [256]:
tags[:50]

,# time (in ms),tag text,adjusted_timestamp,rounded_timestamp
0,0,53_nikola_bilyk,2023-07-09 20:30:30.600,2023-07-09 20:30:30.600
2,2892,6_harald_reinkind,2023-07-09 20:30:33.492,2023-07-09 20:30:33.500
4,5147,18_niclas_ekberg,2023-07-09 20:30:35.747,2023-07-09 20:30:35.750
6,6713,6_harald_reinkind,2023-07-09 20:30:37.313,2023-07-09 20:30:37.300
8,10532,53_nikola_bilyk,2023-07-09 20:30:41.132,2023-07-09 20:30:41.150
10,12750,21_eric_johansson,2023-07-09 20:30:43.350,2023-07-09 20:30:43.350
12,15141,33_kay_smits,2023-07-09 20:30:45.741,2023-07-09 20:30:45.750
14,17128,7_magnus_landin_jacobsen,2023-07-09 20:30:47.728,2023-07-09 20:30:47.750
16,18455,26_johan_hansen,2023-07-09 20:30:49.055,2023-07-09 20:30:49.050
18,19899,7_magnus_landin_jacobsen,2023-07-09 20:30:50.499,2023-07-09 20:30:50.500


In [257]:
# Now you can merge the two dataframes based on the adjusted timestamp
merged_df = pd.merge(match_h1, tags, left_on='formatted local time', right_on='rounded_timestamp', how='left')

In [258]:
columns_from_df2 = tags.columns.difference(["# time (in ms)", "adjusted_timestamp"])
for col in columns_from_df2:
    merged_df[col] = merged_df[col].ffill()

In [259]:
merged_df

,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load,# time (in ms),tag text,adjusted_timestamp,rounded_timestamp
0,1694111429700,2023-07-09 20:30:29.700,43591,214,4,Johannes Golla,876390,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,1694111429700,2023-07-09 20:30:29.700,43603,1104,20,Kevin Møller,592178,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,1694111429700,2023-07-09 20:30:29.700,43632,1522,26,Johan Hansen,1340010,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,1694111429700,2023-07-09 20:30:29.700,43679,1548,21,Eric Johansson,1560210,2,THW Kiel,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,1694111429700,2023-07-09 20:30:29.700,50453,1833,2,Simon Pytlick,1560268,1,SG Flensburg-Handewitt,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545524,1694113910350,2023-07-09 21:11:50.350,43633,1810,27,Aksel Horgen,1611730,1,SG Flensburg-Handewitt,-17.070,7.066,1.6,1.253,-33.848,-0.330,632.104,3.819,Forward,NaN,0.805,NaN,28_eduardo_gurbindo_martinez,NaT,2023-07-09 20:50:21.200
545525,1694113910350,2023-07-09 21:11:50.350,43677,333,17,Patrick Wiencek,125232,2,THW Kiel,-13.753,1.148,1.6,0.364,113.011,-0.342,2024.731,1.186,Left,NaN,0.380,NaN,28_eduardo_gurbindo_martinez,NaT,2023-07-09 20:50:21.200
545526,1694113910350,2023-07-09 21:11:50.350,43618,1832,25,Lukas Joergensen,1515763,1,SG Flensburg-Handewitt,-12.089,3.035,1.6,0.297,NaN,-0.212,534.310,0.853,NaN,NaN,0.700,NaN,28_eduardo_gurbindo_martinez,NaT,2023-07-09 20:50:21.200
545527,1694113910350,2023-07-09 21:11:50.350,43674,1546,16,Tomas Mrkva,124259,2,THW Kiel,-17.723,4.201,1.6,0.520,64.045,-1.543,1256.772,0.777,Forward,1208.0,3.143,NaN,28_eduardo_gurbindo_martinez,NaT,2023-07-09 20:50:21.200


Plan, um Zeit zu überprüfen:
- Zeitpunkt in Videodaten finden, in denen was besonderes stattfindet, bspw.:
    - Ball ist auf dem Boden und ein Spieler in Ballbesitz, der es normalerweise nicht ist
    - Ball berührt den Boden kurz nachdem er einen Moment sehr hoch war
- Zeitpunkt in Daten mit besonderem Tag taggen, in dem exakt der Ball den Boden berührt
- Zeitpunkt in Tabellendaten finden, in denen das stattfindet
- mit gemergten Daten vergleichen, ob zum richtigen Zeitpunkt der Ball den Boden berührt

In [260]:
# find index where first check-tag is used
merged_df[merged_df["tag text"] == " 28_eduardo_gurbindo_martinez"].index[0]

277257

In [262]:
merged_df[276500:279000][(~np.isnan(merged_df["ball possession (id of possessed ball)"])) | (merged_df["group name"] == "Ball")]

C:\Users\Zalipski\AppData\Local\Temp\ipykernel_20356\2538930527.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged_df[276500:279000][(~np.isnan(merged_df["ball possession (id of possessed ball)"])) | (merged_df["group name"] == "Ball")]


,ts in ms,formatted local time,sensor id,mapped id,number,full name,league id,group id,group name,x in m,y in m,z in m,speed in m/s,direction of movement in deg,acceleration in m/s2,total distance in m,metabolic power in W/kg,"player orientation category (forward, back, left, right)",ball possession (id of possessed ball),acceleration load,# time (in ms),tag text,adjusted_timestamp,rounded_timestamp
276500,1694112618650,2023-07-09 20:50:18.650,43696,1728,71,Elias Ellefsen á Skipagøtu,1890154,2,THW Kiel,6.479,2.495,1.600,3.130,-16.760,1.404,26.950,22.862,Forward,1209.0,4.447,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276502,1694112618650,2023-07-09 20:50:18.650,43672,327,6,Harald Reinkind,604492,2,THW Kiel,9.590,-4.328,1.600,2.726,-57.011,0.616,682.864,13.751,Right,1209.0,3.082,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276510,1694112618700,2023-07-09 20:50:18.700,314405,1209,99,Ball3 Ball,FlensBall3,3,Ball,6.944,2.023,1.397,6.972,92.394,-54.476,662.795,NaN,NaN,NaN,NaN,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276515,1694112618700,2023-07-09 20:50:18.700,43696,1728,71,Elias Ellefsen á Skipagøtu,1890154,2,THW Kiel,6.658,2.429,1.600,3.134,-18.863,1.214,26.950,21.065,Forward,1209.0,4.492,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276517,1694112618700,2023-07-09 20:50:18.700,43672,327,6,Harald Reinkind,604492,2,THW Kiel,9.655,-4.482,1.600,2.763,-60.611,0.616,682.864,13.751,Right,1209.0,3.052,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276525,1694112618750,2023-07-09 20:50:18.750,314405,1209,99,Ball3 Ball,FlensBall3,3,Ball,7.042,2.226,1.271,4.724,89.323,-26.205,662.795,NaN,NaN,NaN,NaN,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276530,1694112618750,2023-07-09 20:50:18.750,43696,1728,71,Elias Ellefsen á Skipagøtu,1890154,2,THW Kiel,6.838,2.362,1.600,3.134,-21.053,1.025,26.950,19.268,Forward,1209.0,4.522,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276532,1694112618750,2023-07-09 20:50:18.750,43672,327,6,Harald Reinkind,604492,2,THW Kiel,9.719,-4.635,1.600,2.867,-61.562,0.616,682.864,13.751,Right,1209.0,3.027,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276540,1694112618800,2023-07-09 20:50:18.800,314405,1209,99,Ball3 Ball,FlensBall3,3,Ball,7.258,2.220,1.208,4.497,86.210,-6.335,662.795,NaN,NaN,NaN,NaN,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650
276545,1694112618800,2023-07-09 20:50:18.800,43696,1728,71,Elias Ellefsen á Skipagøtu,1890154,2,THW Kiel,7.015,2.301,1.600,3.134,-23.248,0.812,26.950,17.412,Forward,1209.0,4.537,NaN,7_magnus_landin_jacobsen,NaT,2023-07-09 20:50:04.650


## Show unique timestamps

In [289]:
match_h1["formatted local time"].value_counts()[match_h1["formatted local time"].value_counts() == 1]

formatted local time
2023-07-09 20:46:32.750    1
2023-07-09 20:46:50.300    1
2023-07-09 20:43:04.650    1
2023-07-09 20:45:05.850    1
2023-07-09 20:42:27.550    1
2023-07-09 20:43:04.150    1
2023-07-09 20:43:09.200    1
2023-07-09 20:43:46.300    1
2023-07-09 20:41:41.350    1
2023-07-09 20:41:41.400    1
2023-07-09 20:43:29.300    1
2023-07-09 21:07:27.050    1
2023-07-09 20:44:53.150    1
2023-07-09 20:40:49.050    1
2023-07-09 20:55:16.850    1
2023-07-09 20:55:55.000    1
2023-07-09 20:49:27.150    1
Name: count, dtype: int64

In [263]:
# Group by 'timestamp' and filter groups where all 'ball_possession' values are NaN
timestamps_with_no_possession = match_h1_start.groupby('formatted local time').filter(lambda x: x['ball possession (id of possessed ball)'].isna().all())

# Extract the unique timestamps where no player has possession of the ball
unique_timestamps = timestamps_with_no_possession['formatted local time'].nunique()

unique_timestamps_total = match_h1_start["formatted local time"].unique()

print(unique_timestamps)
print(unique_timestamps_total)

13539
<DatetimeArray>
['2023-07-09 20:30:29.800000', '2023-07-09 20:30:29.850000',
 '2023-07-09 20:30:29.900000', '2023-07-09 20:30:29.950000',
        '2023-07-09 20:30:30', '2023-07-09 20:30:30.050000',
 '2023-07-09 20:30:30.100000', '2023-07-09 20:30:30.150000',
 '2023-07-09 20:30:30.200000', '2023-07-09 20:30:30.250000',
 ...
 '2023-07-09 21:11:49.900000', '2023-07-09 21:11:49.950000',
        '2023-07-09 21:11:50', '2023-07-09 21:11:50.050000',
 '2023-07-09 21:11:50.100000', '2023-07-09 21:11:50.150000',
 '2023-07-09 21:11:50.200000', '2023-07-09 21:11:50.250000',
 '2023-07-09 21:11:50.300000', '2023-07-09 21:11:50.350000']
Length: 36564, dtype: datetime64[ns]
